In [11]:
from __future__ import print_function, division
from future.utils import iteritems
from builtins import range, input
import math

In [1]:
import networkx as nx
import nltk
import numpy as np
import matplotlib.pyplot as plt
from nltk.stem import WordNetLemmatizer
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE, LocallyLinearEmbedding as LLE
from sklearn.feature_extraction.text import TfidfTransformer


In [2]:
from Normalization import Normalization

In [3]:
 nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\serena\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\serena\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [4]:
wordnet_lemmatizer = WordNetLemmatizer()

titles = [line.rstrip() for line in open('all_book_titles.txt')]

# copy tokenizer from sentiment example
stopwords = set(w.rstrip() for w in open('stopwords.txt'))
# add more stopwords specific to this problem
stopwords = stopwords.union({
    'introduction', 'edition', 'series', 'application',
    'approach', 'card', 'access', 'package', 'plus', 'etext',
    'brief', 'vol', 'fundamental', 'guide', 'essential', 'printed',
    'third', 'second', 'fourth', })
def my_tokenizer(s):
    s = s.lower() # downcase
    tokens = nltk.tokenize.word_tokenize(s) # split string into words (tokens)
    tokens = [t for t in tokens if len(t) > 2] # remove short words, they're probably not useful
    tokens = [wordnet_lemmatizer.lemmatize(t) for t in tokens] # put words into base form
    tokens = [t for t in tokens if t not in stopwords] # remove stopwords
    tokens = [t for t in tokens if not any(c.isdigit() for c in t)] # remove any digits, i.e. "3rd edition"
    return tokens


# create a word-to-index map so that we can create our word-frequency vectors later
# let's also save the tokenized versions so we don't have to tokenize again later
word_index_map = {}
current_index = 0
all_tokens = []
all_titles = []
index_word_map = []
print("num titles:", len(titles))
print("first title:", titles[0])
for title in titles:
    try:
        title = title.encode('ascii', 'ignore') # this will throw exception if bad characters
        title = title.decode('utf-8')
        all_titles.append(title)
        tokens = my_tokenizer(title)
        all_tokens.append(tokens)
        for token in tokens:
            if token not in word_index_map:
                word_index_map[token] = current_index
                current_index += 1
                index_word_map.append(token)
    except Exception as e:
        print(e)



# now let's create our input matrices - just indicator variables for this example - works better than proportions
def tokens_to_vector(tokens):
    x = np.zeros(len(word_index_map))
    for t in tokens:
        i = word_index_map[t]
        x[i] += 1
    return x

N = len(all_tokens)
D = len(word_index_map)
X = np.zeros((D, N)) # terms will go along rows, documents along columns
i = 0
for tokens in all_tokens:
    X[:,i] = tokens_to_vector(tokens)
    i += 1

def d(u, v):
    diff = u - v
    return diff.dot(diff)

def cost(X, R, M):
    cost = 0
    for k in range(len(M)):
        # method 1
        # for n in range(len(X)):
        #     cost += R[n,k]*d(M[k], X[n])

        # method 2
        diff = X - M[k]
        sq_distances = (diff * diff).sum(axis=1)
        cost += (R[:,k] * sq_distances).sum()
    return cost

def plot_k_means(X, K, index_word_map, max_iter=20, beta=1.0, show_plots=True):
    N, D = X.shape
    M = np.zeros((K, D))
    R = np.zeros((N, K))
    exponents = np.empty((N, K))

    # initialize M to random
    for k in range(K):
        M[k] = X[np.random.choice(N)]

    costs = np.zeros(max_iter)
    for i in range(max_iter):
        # step 1: determine assignments / resposibilities
        # is this inefficient?
        for k in range(K):
            for n in range(N):
                # R[n,k] = np.exp(-beta*d(M[k], X[n])) / np.sum( np.exp(-beta*d(M[j], X[n])) for j in range(K) )
                exponents[n,k] = np.exp(-beta*d(M[k], X[n]))

        R = exponents / exponents.sum(axis=1, keepdims=True)

        # step 2: recalculate means
        for k in range(K):
            M[k] = R[:,k].dot(X) / R[:,k].sum()

        costs[i] = cost(X, R, M)
        if i > 0:
            if np.abs(costs[i] - costs[i-1]) < 10e-5:
                break

    if show_plots:
        # plt.plot(costs)
        # plt.title("Costs")
        # plt.show()

        random_colors = np.random.random((K, 3))
        colors = R.dot(random_colors)
        plt.figure(figsize=(80.0, 80.0))
        plt.scatter(X[:,0], X[:,1], s=300, alpha=0.9, c=colors)
        annotate1(X, index_word_map)
        # plt.show()
        plt.savefig("test.png")


    # print out the clusters
    hard_responsibilities = np.argmax(R, axis=1) # is an N-size array of cluster identities
    # let's "reverse" the order so it's cluster identity -> word index
    cluster2word = {}
    for i in range(len(hard_responsibilities)):
      word = index_word_map[i]
      cluster = hard_responsibilities[i]
      if cluster not in cluster2word:
        cluster2word[cluster] = []
      cluster2word[cluster].append(word)

    # print out the words grouped by cluster
    for cluster, wordlist in cluster2word.items():
      print("cluster", cluster, "->", wordlist)

    return M, R


num titles: 2373
first title: Philosophy of Sex and Love A Reader


In [5]:
def plot_k_means11(X, K, m,max_iter=20, beta=1.0, show_plots=False):
    N, D = X.shape
    # R = np.zeros((N, K))
    exponents = np.empty((N, K))

    # initialize M to random
    initial_centers = np.random.choice(N, K, replace=False)
    M = m

    costs = []
    k = 0
    for i in range(max_iter):
        k += 1
        # step 1: determine assignments / resposibilities
        # is this inefficient?
        for k in range(K):
            for n in range(N):
                exponents[n,k] = np.exp(-beta*d(M[k], X[n]))
        R = exponents / (exponents.sum(axis=1, keepdims=True)+0.000000000001)


        # step 2: recalculate means
        # decent vectorization
        # for k in range(K):
        #     M[k] = R[:,k].dot(X) / R[:,k].sum()
        # oldM = M

        # full vectorization
        M = R.T.dot(X) / R.sum(axis=0, keepdims=True).T
        # print("diff M:", np.abs(M - oldM).sum())

        c = cost(X, R, M)
        costs.append(c)
        if i > 0:
            if np.abs(costs[-1] - costs[-2]) < 1e-5:
                break

        if len(costs) > 1:
            if costs[-1] > costs[-2]:
                pass
                # print("cost increased!")
                # print("M:", M)
                # print("R.min:", R.min(), "R.max:", R.max())

    if show_plots:
        plt.plot(costs)
        plt.title("Costs")
        plt.show()

        random_colors = np.random.random((K, 3))
        colors = R.dot(random_colors)
        plt.scatter(X[:,0], X[:,1], c=colors)
        plt.show()

    #print("Final cost", costs[-1])
    return M, R


In [6]:
def plot_k_means2(X, K, max_iter=20, beta=1.0, show_plots=False):
    N, D = X.shape
    # R = np.zeros((N, K))
    exponents = np.empty((N, K))

    # initialize M to random
    initial_centers = np.random.choice(N, K, replace=False)
    M = X[initial_centers]

    costs = []
    k = 0
    for i in range(max_iter):
        k += 1
        # step 1: determine assignments / resposibilities
        # is this inefficient?
        for k in range(K):
            for n in range(N):
                exponents[n,k] = np.exp(-beta*d(M[k], X[n]))
        R = exponents / exponents.sum(axis=1, keepdims=True)


        # step 2: recalculate means
        # decent vectorization
        # for k in range(K):
        #     M[k] = R[:,k].dot(X) / R[:,k].sum()
        # oldM = M

        # full vectorization
        M = R.T.dot(X) / R.sum(axis=0, keepdims=True).T
        # print("diff M:", np.abs(M - oldM).sum())

        c = cost(X, R, M)
        costs.append(c)
        if i > 0:
            if np.abs(costs[-1] - costs[-2]) < 1e-5:
                break

        if len(costs) > 1:
            if costs[-1] > costs[-2]:
                pass
                # print("cost increased!")
                # print("M:", M)
                # print("R.min:", R.min(), "R.max:", R.max())

    if show_plots:
        plt.plot(costs)
        plt.title("Costs")
        plt.show()

        random_colors = np.random.random((K, 3))
        colors = R.dot(random_colors)
        plt.scatter(X[:,0], X[:,1], c=colors)
        plt.show()
    return M, R

In [9]:
def plot_tri_k_means(X, K1, index_word_map, max_iter=20, beta=1.0, show_plots=True):
    
    X = Normalization(X)
    
    K = int(math.sqrt(K1))
    M,R = plot_k_means2(X,K,beta=20,show_plots=False)
    
    ######### Trilevel  Fully divided ###########       
    
    label = np.argmax(R,axis=1)
    gnum = K
    Group = []

    for i in range(gnum):
        index = np.where(label==i)
        print(np.asarray(X[index,:]).shape)
        x,y,z = np.asarray(X[index,:]).shape
        Group.append(np.asarray(X[index,:]).reshape((y,z)))
    
    #comupute the standard diviation 
    std = []
    
    for i in range(len(Group)):
        std.append(np.average(np.std(Group[i],axis=1)))


    #compute the Kc
    KC = []
    k = []
    for i in range(gnum):
        KC.append(Group[i].shape[1]*std[i])

        
    for i in range(gnum):
        k.append (round(KC[i]/(sum(KC))*K1)) 
        
    k[-1] = K1 - sum(k[:-1])
    
    M = []
    R = []
    
    # subclass
    for i in range(gnum):
        M1,R1 = plot_k_means2(Group[i],int(k[i]),beta=20,show_plots=False)
        M.append(M1)
        R.append(R1)
    
    Mnew = np.ones((1,z))
    for i in range(gnum):
        Mnew = np.concatenate((Mnew,M[i]),axis=0)
    Mnew = Mnew[1:,:]
    
    MM,RR = plot_k_means11(X,K1,Mnew,beta=20)
    
    if show_plots:
        
        random_colors = np.random.random((K1, 3))
        colors = RR.dot(random_colors)
        plt.figure(figsize=(80.0, 80.0))
        plt.scatter(X[:,0], X[:,1], s=300, alpha=0.9, c=colors)
        annotate1(X, index_word_map)
        # plt.show()
        plt.savefig("test.png")

    # print out the clusters
    """
      hard_responsibilities = np.argmax(R, axis=1) # is an N-size array of cluster identities
        # let's "reverse" the order so it's cluster identity -> word index
        cluster2word = {}
        for i in range(len(hard_responsibilities)):
          word = index_word_map[i]
          cluster = hard_responsibilities[i]
          if cluster not in cluster2word:
            cluster2word[cluster] = []
          cluster2word[cluster].append(word)

        # print out the words grouped by cluster
        for cluster, wordlist in cluster2word.items():
          print("cluster", cluster, "->", wordlist)
      
   """

    return MM, RR



In [8]:
def annotate1(X, index_word_map, eps=0.1):
  N, D = X.shape
  placed = np.empty((N, D))
  for i in range(N):
    x, y = X[i]

    # if x, y is too close to something already plotted, move it
    close = []

    x, y = X[i]
    for retry in range(3):
      for j in range(i):
        diff = np.array([x, y]) - placed[j]

        # if something is close, append it to the close list
        if diff.dot(diff) < eps:
          close.append(placed[j])

      if close:
        # then the close list is not empty
        x += (np.random.randn() + 0.5) * (1 if np.random.rand() < 0.5 else -1)
        y += (np.random.randn() + 0.5) * (1 if np.random.rand() < 0.5 else -1)
        close = [] # so we can start again with an empty list
      else:
        # nothing close, let's break
        break

    placed[i] = (x, y)

    plt.annotate(
      s=index_word_map[i],
      xy=(X[i,0], X[i,1]),
      xytext=(x, y),
      arrowprops={
        'arrowstyle' : '->',
        'color' : 'black',
      }
    )

print("vocab size:", current_index)

transformer = TfidfTransformer()
X = transformer.fit_transform(X).toarray()

reducer = TSNE()
Z = reducer.fit_transform(X)
#plot_tri_k_means(Z[:,:2], current_index//10, index_word_map, show_plots=True)

vocab size: 2069


In [ ]:
plot_tri_k_means(Z[:,:2], current_index//10, index_word_map, show_plots=True)

(1, 185, 2)
(1, 95, 2)
(1, 217, 2)
(1, 189, 2)
(1, 180, 2)
(1, 73, 2)
(1, 164, 2)
(1, 213, 2)
(1, 144, 2)
(1, 128, 2)
(1, 68, 2)
(1, 168, 2)
(1, 173, 2)
(1, 72, 2)


(array([[0.36288225, 0.43889597],
        [0.3628559 , 0.43889455],
        [0.36288142, 0.43889587],
        [0.36289044, 0.43889643],
        [0.36284516, 0.43889422],
        [0.36284507, 0.43889431],
        [0.3628862 , 0.43889615],
        [0.36287678, 0.4388957 ],
        [0.36287973, 0.43889591],
        [0.6415657 , 0.5173216 ],
        [0.64156348, 0.51731893],
        [0.64156315, 0.51731853],
        [0.64155775, 0.51731208],
        [0.64156054, 0.51731545],
        [0.64156243, 0.51731768],
        [0.6415574 , 0.51731167],
        [0.64155956, 0.51731425],
        [0.64155841, 0.51731287],
        [0.64156163, 0.51731673],
        [0.51222342, 0.73464577],
        [0.51222191, 0.73464435],
        [0.5122115 , 0.73464766],
        [0.51222887, 0.73464368],
        [0.51223011, 0.73464491],
        [0.51224963, 0.73464195],
        [0.5122385 , 0.73464234],
        [0.51224364, 0.73464204],
        [0.51222128, 0.73464521],
        [0.51221788, 0.73464533],
        [0.512